<div class='heading'>
    <div style='float:left;'><h1>CPSC 4300/6300: Applied Data Science</h1></div>
    <img style="float: right; padding-right: 10px; width: 65px" src="https://raw.githubusercontent.com/bsethwalker/clemson-cs4300/main/images/clemson_paw.png"> </div>

## Week 13 | Lab: Building and Regularizing a Neural Network 

**Clemson University** </br>
**Instructor(s):** Tim Ransom </br>

------------------------------------------------------------------------
## Learning objectives

- Explain the concept of overfitting in neural networks.
- Apply regularization techniques like dropout to prevent overfitting.
- Analyze the impact of different regularization methods on model performance.
- Interpret training and validation curves to diagnose overfitting.
- Tune hyperparameters to optimize the performance of a neural network.

### INSTRUCTIONS

-   As much as possible, try and stick to the hints and functions we
    import at the top of the homework, as those are the ideas and tools
    the class supports and are aiming to teach. And, if a problem
    specifies a particular library, you're required to use that library,
    and possibly others from the import list.
-   Please use .head() when viewing data. 

In [ ]:
""" RUN THIS CELL TO GET THE RIGHT FORMATTING """
import requests
from IPython.core.display import HTML
css_file = 'https://raw.githubusercontent.com/bsethwalker/clemson-cs4300/main/css/cpsc6300.css'
styles = requests.get(css_file).text
HTML(styles)


In [ ]:
import random
random.seed(112358)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotcheck.base import PlotTester
%matplotlib inline

import seaborn as sns
pd.set_option('display.width', 1500)
pd.set_option('display.max_columns', 100)

# from sklearn.utils import shuffle


In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf

print(tf.__version__)

<div class='theme'> Neural Networks: Learning Fourier Transform Mapping </div>

Neural networks are, of course, a large and complex topic that cannot be covered in a single exercise. Here, we'll focus on the key idea of Artificial Neural Networks (ANNs): they can learn a mapping from example input data $ X $ (of fixed size) to example output data $ Y $ (of fixed size). This is the same concept as every other classification and regression task we've learned so far.

In this exercise, we'll explore how neural networks generalize and what patterns they learn. Specifically, we will see if neural networks can learn a limited version of the **Fourier Transform**.

---

## What is a Fourier Transform?

The Fourier Transform takes in values from some function and returns a set of sine and cosine functions that, when added together, approximate the original function. 

In mathematical terms:
$
F(s) = \int_{-\infty}^{\infty} f(x)e^{-i x s} dx
$

### Key Concept:
The transformed function at some point $ s $ measures, in some sense, how much the original $ f(x) $ resembles a wave with a period of $ s $.

### Example:
Let $ f(x) = 4\cos(x) + \sin(2x) $.  
The Fourier Transform, $ \mathcal{F}(s) $, would be 0 everywhere except at -2, -1, 1, and 2, corresponding to waves of periods 1 and 1/2. The values at these points represent the magnitude of the waves and their phases (e.g., sine waves vs cosine waves).

---

## Problem Setup

We will train a neural network to approximate a limited Fourier Transform. The network will map **1000 sample values** from a function (equally spaced along $ 0 $ to $ 10\pi $ ) to **four features** representing the sine and cosine components of that function.

### Dataset Details:
- **Input**: A 1000-entry vector (samples from the original function).
- **Output**: A 4-entry vector (features of sine and cosine components).

### Dataset Shapes:
- $ X_{\text{train}} $: Shape $ N \times 1000 $
- $ y_{\text{train}} $: Shape $ N \times 4 $

---

## Datasets

We will use 6 datasets in this exercise:

1. **Training Data**:
   - `sinewaves_X_train.npy`: Shape $ 10,000 \times 1,000 $
   - `sinewaves_y_train.npy`: Shape $ 10,000 \times 4 $

   *Generated by randomly selecting $ a, b, c, d $ in the interval $[0,1]$ and building the curve $ a\sin(bx) + c\cos(dx) $.*

2. **Test Data**:
   - `sinewaves_X_test.npy`: Shape $ 2,000 \times 1,000 $
   - `sinewaves_y_test.npy`: Shape $ 2,000 \times 4 $

   *Generated similarly to the training data.*

3. **Extended Test Data**:
   - `sinewaves_X_extended_test.npy`: Shape $ 9 \times 1,000 $
   - `sinewaves_y_extended_test.npy`: Shape $ 9 \times 4 $

   *Tests whether the network can generalize beyond the training data (e.g., to negative values of $ a $).*

---

## Objective

The goal is to train a neural network to map from the input $ X $ (1000-entry vector) to the output $ y $ (4-entry vector). The network should generalize well to unseen data.

---

### Questions 1.1 and 1.2:
- Get familiar with the format of the data.

---

## Hints

1. Use TensorFlow's `tf.keras.Sequential` model for this exercise.
   - Documentation: [TensorFlow Sequential Model](https://www.tensorflow.org/guide/keras/sequential_model)

2. A strong model should achieve:
   - **Validation Error**: ~0.03
   - **Excellent Model**: Validation error of ~0.02.

# Function: `plot_predictions`

## Purpose:
The `plot_predictions` function evaluates and visualizes the performance of a neural network trained to approximate Fourier Transform parameters. It compares the true sine curves with those implied by the model's predictions.

## Parameters:
- `model`: A TensorFlow/Keras model trained to map inputs of shape $(n, 1000$) to outputs of shape $(n, 4)$.
- `test_x`: A test input dataset of shape $(n, 1000)$, representing sampled sine wave data.
- `test_y`: A test output dataset of shape $(n, 4)$, representing the true Fourier Transform parameters.
- `count` (optional): The number of examples to plot. If not provided:
  - Plots all examples if the dataset has fewer than 20 examples.
  - Otherwise, plots 5 randomly selected examples.

---

## Functionality:
1. **Helper Function: `y2x`**:
   - Takes an $(n, 4)$ output array and reverse-engineers the sine curves described by those Fourier Transform parameters.
   - **Parameters:**
     - $a$, $b$, $c$, $d$: Extracted from the model's predictions.
   - **Output**:
     - Reconstructed sine wave data for each test example using:
       $
       x_{\text{data}} = a \sin(b x) + c \cos(d x)
       $

2. **Visualization**:
   - Plots both the true sine wave (`test_x`) and the sine wave implied by the model's predictions for each selected example.
   - Displays a maximum of `count` examples (default: 5 if $\text{test\_x.shape[0]} > 20$).
   - Sets plot limits to keep the curves visually consistent.

3. **Output**:
   - Prints the true Fourier Transform parameters (`test_y`) and the corresponding predictions (`predicted`) for each plotted example.

---

## Example Usage:
```python
# Assuming `model` is a trained neural network, and test datasets are available
plot_predictions(model, sinewaves_X_test, sinewaves_y_test)


In [ ]:
def plot_predictions(model, test_x, test_y, count=None):
    # Model - a tf.keras model that takes in (n,1000) training data and predicts (n,4) output data
    # test_x - a (n,1000) input dataset
    # test_y - a (n,4) output dataset
    # This function will plot the sine curves in the training data and those implied by the model's predictions.
    # It will also print the predicted and actual output values.

    #helper function that takes the n by 4 output and reverse-engineers 
    #the sine curves that output would create
    def y2x(y_data):
        #extract parameters
        a=y_data[:,0].reshape(-1,1)
        b=y_data[:,1].reshape(-1,1)
        c=y_data[:,2].reshape(-1,1)
        d=y_data[:,3].reshape(-1,1)

        #build the matching training data
        x_points = np.linspace(0,10*np.pi,1000)
        x_data = a*np.sin(np.outer(b,x_points)) + c*np.cos(np.outer(d,x_points))
        return x_data

    #if <20 examples, plot all. If more, just plot 5
    if count==None:
        if test_x.shape[0]>20:
            count=5
        else:
            count=test_x.shape[0]

    #build predictions
    predicted = model.predict(test_x)
    implied_x = y2x(predicted)
    for i in range(count):
        plt.plot(test_x[i,:],label='true')
        plt.plot(implied_x[i,:],label='predicted')
        plt.legend()
        plt.ylim(-2.1,2.1)
        plt.xlabel("x value")
        plt.xlabel("y value")
        plt.title("Curves using the Neural Network's Approximate Fourier Transform")
        plt.show()
        print("true:", test_y[i,:])
        print("predicted:", predicted[i,:])

In [ ]:
X_train = np.load('data/sinewaves_dataset/sinewaves_X_train.npy')
y_train = np.load('data/sinewaves_dataset/sinewaves_y_train.npy')

X_test = np.load('data/sinewaves_dataset/sinewaves_X_test.npy')
y_test = np.load('data/sinewaves_dataset/sinewaves_y_test.npy')

X_extended_test = np.load('data/sinewaves_dataset/sinewaves_X_extended_test.npy')
y_extended_test = np.load('data/sinewaves_dataset/sinewaves_y_extended_test.npy')


<hr style='height:2pt'>

<div class='exercise'> <b> Exercise 1 : Plot the Sinusoidal Curve</b> </div>

#### Objective:
Visualize the first row of the `X_train` training data to verify that it represents a sinusoidal curve.

---

#### Instructions:

1. **Dataset is Loaded in `X_train`**:

2. **Plot the First Row of `X_train`**:
   - Use `matplotlib` to plot the first row of `X_train` (i.e., `X_train[0]`).
   - Use `plt.subplots()` to create the figure and axis (`fig`, `ax`).
   - Plot the data using `ax.plot()`.

3. **Customize the Plot**:
      - Title: **Sinusoidal Curve: First Row of X_train**
      - X-axis label: **Sample Index**
      - Y-axis label: **"Value"**
      - Add a legend to indicate what is being plotted.
      - Add gridlines for better readability.

---

In [ ]:
"""Write your code for exercise 1 here:"""

# your code here
raise NotImplementedError

In [ ]:
####################################
###✅🔒 Hidden tests here ✅🔒### 
###################################

<div class='exercise'> <b> Exercise 2: Verify the Sinusoidal Function</b> </div>

## Objective:
Verify that the first row of the `X_train` data represents 1000 equally spaced points in $[0, 10\pi]$ from the function:
$
f(x) = 0.024 \sin(0.533x) + 0.018 \cos(0.558x)
$

---

## Instructions:

1. **Ensure the Datasets are Loaded**:
   - Check if the `sinewaves_X_train.npy` and `sinewaves_y_train.npy` datasets are loaded into `X_train` and `y_train`.

2. **Extract Parameters from `y_train`**:
   - The first row of `y_train` provides the sine and cosine parameters:
     - $a = 0.024$
     - $b = 0.533$
     - $c = 0.018$
     - $d = 0.558$

3. **Generate the Function**:
   - Use below formula to generate sine-cosine function `f_x`:
     $
     f(x) = a \sin(bx) + c \cos(dx)
     $
   - Generate 1000 equally spaced points in $[0, 10\pi]$ and store to `x_points` in order to match the input space of `X_train`.
   
4. **Plot the first row of `X_train` and the generated `f(x)` function on the **same figure** using `matplotlib`.**
      - Use `plt.subplots()` to create `fig` and `ax`.
      - Plot the data from `X_train[0]` (the true data) and the curve `f(x)`(i.e `f_x`) using distinct colors or line styles.
      - Add title: **"Verification of the Sinusoidal Function"**
      - Label the axes: **"x"** and **"Value"**
      - Add a legend and grid for clarity.
      - Check if both curves overlap closely.


---

In [ ]:
"""Write your code for exercise 2 here:"""

# your code here
raise NotImplementedError

In [ ]:
####################################
###✅🔒 Hidden tests here ✅🔒### 
###################################

<div class='theme'> <b> Exercise 3 :</b> Use <b>tf.keras</b> to build a fully-connected neural network </div>

<div class='exercise'> <b> Exercise 3.1 </b> </div>

## Objective: 

In this task, you will create a feedforward neural network using TensorFlow's `tf.keras` module.

## Instructions:

- Use `tf.keras.models.Sequential` to define the model architecture.
- Use `tf.keras.layers.Dense` to add fully-connected (dense) layers.
- Your model should:
  - Accept an input of shape **(1000,)** (since each training example has 1000 features).
  - Have at least one or more hidden layers with any number of nodes.
  - Use `'relu'` activation for hidden layers.
  - End with a final output layer of **4 units** (to match the shape of `y_train`), without an activation function.

**Store your model in a variable named `model`.**

You can experiment with the number of layers and neurons per layer to optimize performance.

In [ ]:
"""Write your code for exercise 3.1 here:"""

# your code here
raise NotImplementedError

In [ ]:
####################################
###✅🔒 Hidden tests here ✅🔒### 
###################################


<div class='exercise'> <b> Exercise 3.2: Compile and Summarize Your Neural Network</b> </div>

**Now that we have defined your fully-connected neural network, the next step is to compile it for training.**

## Instructions:

- Compile your model using:
  - **Loss function**: `'mean_absolute_error'` (MAE)
  - **Optimizer**: `'adam'`
- Use the `.compile()` method from Keras and store the compiled model in the same variable `model`.
- After compiling, display the model architecture using `.summary()`.

In [ ]:
"""Write your code for exercise 3.2 here:"""

# your code here
raise NotImplementedError

In [ ]:
####################################
###✅🔒 Hidden tests here ✅🔒### 
###################################

<div class="theme"> Question 1:</div>

**Why does the first hidden layer in a fully connected neural network with input size 1000 and 64 neurons have 64,064 trainable parameters?**

1. Because each input connects to each neuron, totaling $1000\times 64$ and there are 64 bias terms — one per neuron.
2. Because it’s a convolutional layer with filters and stride, contributing additional parameters.
3. Because 64 neurons each connect to 64 inputs, and there are no bias terms.
4. Because the output layer influences the number of parameters in the first layer.

**Store your answer in an integer variable named `answer` in the code cell below.**


In [ ]:
"""Your answer here:"""
# your code here
raise NotImplementedError

In [ ]:
####################################
###✅🔒 Hidden checks here ✅🔒### 
###################################

<div class='exercise'> <b> Exercise 3.3: Train Your Neural Network </b> </div>

Now that your model is compiled, it's time to train it using the training data.

## Instructions:
- Fit your model using the `.fit()` method with the following configuration:
  - **Epochs**: `50`
  - **Batch size**: `32`
  - **Validation split**: `0.2` (this will automatically split 20% of the training data for validation)

- Store the returned training history object to variable named `model_history`.
    - eg. code
        ```python
        model_history = model.fit(......)
        ```
        
**This history object will contain the loss and validation loss over epochs, which can later be used to plot learning curves and diagnose overfitting.**

**Note:** You can train for longer if you wish to further reduce the loss – the fit tends to improve over time.

In [ ]:
"""Write your code for exercise 3.3 here:"""

# your code here
raise NotImplementedError

In [ ]:
####################################
###✅🔒 Hidden tests here ✅🔒### 
###################################

<div class='exercise'> <b> Exercise 4: Evaluate Your Neural Network on Unseen Data </b> </div>

## Objective: 

In this exercise, you'll evaluate how well your trained model generalizes to new, unseen data.

## Instructions:
- Use the `.evaluate()` method to compute the **loss on the test set** (`X_test`, `y_test`).
- Store the result in a variable called `test_loss` and print it.
- Use the provided `plot_predictions()` function to:
  - Visually compare the **true sinusoidal curve** (from test data) and the **curve reconstructed from the model's predicted Fourier coefficients**.
  - By default, the function plots 5 randomly selected samples if the test set contains more than 20 rows.

In [ ]:
"""Write your code for exercise 4 here:"""

# your code here
raise NotImplementedError

In [ ]:
####################################
###✅🔒 Hidden tests here ✅🔒### 
###################################

<div class="theme"> Question 2:</div>

**Based on the model’s predictions and the resulting plots on the test set, how can you best describe the model’s performance?**

1. The predicted sine curves closely match the actual data, indicating the model has successfully learned the mapping from input samples to Fourier coefficients.
2. The predicted curves show no resemblance to the actual curves, suggesting the model failed to train.
3. The model's predictions look perfect across all examples, proving the model has memorized the training data.
4. The predicted curves are always flat lines, indicating that the model only learned the bias term.

**Store your answer in an integer variable named `answer` in the code cell below.**

In [ ]:
"""Your answer here:"""
# your code here
raise NotImplementedError

In [ ]:
####################################
###✅🔒 Hidden checks here ✅🔒### 
###################################

<div class='exercise'> <b> Exercise 5: Evaluate Model Performance on Extended Test Set </b> </div>

**The extended test set contains `9` special examples generated differently from the original training distribution (e.g., by allowing negative coefficient values). This helps assess how well the model generalizes to edge cases or unseen patterns.**

## Instructions:
- Use the `.evaluate()` method to compute the **loss on the extended test set** (`X_extended_test`, `y_extended_test`) and store it in a variable called `extended_test_loss`.
- Use the `plot_predictions()` function with `count=9` to visualize **all 9 predictions** and compare them with the true values.
- Review the plot and printed values to evaluate and answer below question:
  - Which examples the model **accurately predicts**
  - Which examples the model **struggles to match**

**This test helps assess whether your model generalizes beyond the distribution it was trained on.**

In [ ]:
"""Write your code for exercise 5 here:"""

# your code here
raise NotImplementedError

In [ ]:
####################################
###✅🔒 Hidden tests here ✅🔒### 
###################################

<div class="theme"> Question 3:</div>

**Based on the model’s predictions on the extended test set, which of the following best describes its performance?**

1. The model performs well on all examples, suggesting it generalizes perfectly beyond the training distribution.
2. The model performs poorly on all examples, indicating it failed to learn meaningful patterns.
3. The model performs well on some examples and poorly on others, suggesting it generalizes moderately but struggles with unusual cases.
4. The model performs worse on the extended test set because it has more training data than the original set.

**Store your answer in an integer variable named `answer` in the code cell below.**


In [ ]:
"""Your answer here:"""
# your code here
raise NotImplementedError

In [ ]:
####################################
###✅🔒 Hidden checks here ✅🔒### 
###################################

## Conclusion

**In this lab, you built and trained a fully-connected neural network to approximate a simplified version of the Fourier Transform.**

- Explored how a neural network can learn to map complex input patterns (like sine wave samples) to simpler outputs (Fourier coefficients).
- Visualized training data and verified that it matched expected sine-cosine formulations.
- Designed and compiled a neural network using `tf.keras`, fine-tuned its architecture, and evaluated it using training and test datasets.
- Assessed your model's ability to generalize using a specially crafted extended test set containing edge-case examples.
- Reflected on model performance by analyzing loss values, prediction curves, and potential overfitting.

#### Key Takeaways:
- Neural networks are powerful tools for learning mappings, but their performance is highly dependent on the diversity and representativeness of the training data.
- While your model learned the underlying function reasonably well, it showed limitations when tested on data outside the training distribution.
- Proper **evaluation**, **visualization**, and **reflection** are essential for understanding when and how a model generalizes well.


## Well done on completing this lab! 💪


# END